### Libraries and Dataset

In [1]:
import pandas as pd
import math

In [2]:
dataset_names = ['2019','2015','2010','2000']

In [3]:
count = 1
datasets ={}
for dataset in dataset_names:
    df = pd.read_excel('ghe2019_death-rates-countryfa5d6a0b-0f85-449f-ba5f-ea6fcaeae34c_6f5f17c3-3fae-4660-ad72-56e353b43c55.xlsx',
                  sheet_name = count, skiprows = 6, header = 0)
    datasets[dataset] = df
    count += 1

### Data Cleaning

After looking at the excel file downloaded, we can see there are 8 sheets corresponding to the years and indicators. 

So we have data for 2000, 2010, 2015 and 2019 for the age-standardized death rate and the crude death rate.

For the purpose of this project we decided to focus on the ASDR as it takes into adjusts for differences in ages distribution, allowing for a better comparison between countries.

Therefore, we will keep only 4 sheets of the excel file.

Looking at the information in each sheet, we can see they all follow the same pattern.

Our goal will be to clean the data set until we have the following collumns:

['Disease Group','Disease Group2','Disease Name','Year','Country'Death Rate]

We will start by cleaning the sheets as 4 different separate datasets and group them in the end.


#### Remove unnecessary data

We can see by initial data exploration that diseases and death rates are aggregated into groups with totals for each group.

It is possible then to keep just the value of the smallest subset.

The other death rates can be later obtained by aggregation methods if required

In [4]:
for key, df in datasets.items():
    datasets[key] = df.drop(df[df['Sex'] == 'Males'].index)

In [5]:
for key, df in datasets.items():
    datasets[key] = df.drop(df[df['Sex'] == 'Females'].index)

In [6]:
for key, df in datasets.items():
    datasets[key] = df.dropna(subset = ['Sex'])

In [7]:
for key, df in datasets.items():
    datasets[key] = df.drop(['Member State\n(See Notes for explanation of colour codes)','Sex',
                             'Unnamed: 2', 'Unnamed: 5','GHE cause', 'Unnamed: 4'], axis = 1)

In [8]:
for key, df in datasets.items():
    datasets[key] = df.dropna(subset = ['GHE code'])

#### Unpivot Data and merge dataset

In [9]:
column_list = datasets['2019'].columns.tolist()
countries = column_list[1:]

In [10]:
for key, df in datasets.items():
    datasets[key] = pd.melt(df, id_vars = ['GHE code'],
                 value_vars = countries, var_name = 'Country', value_name = 'Death Rate')

In [11]:
data_frames = [datasets['2000'],datasets['2010'],datasets['2015'],datasets['2019']]
years = ['2000','2010','2015','2019']

In [12]:
for i, df in enumerate(data_frames):
    df['Year'] = years[i]

In [13]:
death_rates = pd.concat(data_frames, ignore_index = True)

#### Get the Disease Group columns

In [14]:
df = pd.read_excel('ghe2019_death-rates-countryfa5d6a0b-0f85-449f-ba5f-ea6fcaeae34c_6f5f17c3-3fae-4660-ad72-56e353b43c55.xlsx',
                  sheet_name = 1, skiprows = 6, header = 0)

In [15]:
df = df.drop(df[df['Sex'] == 'Males'].index)
df = df.drop(df[df['Sex'] == 'Females'].index)
df = df.dropna(subset = ['Sex'])

In [16]:
df = df.iloc[2:, [1,2,3,4,5]]

In [17]:
df['Unnamed: 2'] = df['Unnamed: 2'].fillna(method='ffill')

In [18]:
index_list = ['I.','II.','III.']
names = ['Communicable, maternal, perinatal and nutritional conditions', 'Noncommunicable diseases','Injuries']
Disease_Group = dict(zip(index_list,names))

In [19]:
df['Disease Group'] =df['Unnamed: 2'].map(Disease_Group)

In [20]:
df['GHE cause'] = df['GHE cause'].fillna(method='ffill')

In [21]:
index_conditions = [('I.','A.'),('I.','B.'),('I.','C.'),('I.','D.'),('I.','E.'),
                    ('II.','A.'),('II.','B.'),('II.','C.'),('II.','D.'),('II.','E.'),('II.','F.'),('II.','G.'),
                    ('II.','H.'),('II.','I.'),('II.','J.'),('II.','K.'),('II.','L.'),('II.','M.'),('II.','N.'),
                    ('II.','O.'),('II.','P.'),
                    ('III.','A.'),('III.','B.')]

In [22]:
my_list = list(df['Unnamed: 4'].unique())
name_list = [value for value in my_list if not
            (isinstance(value, float) and math.isnan(value))]
name_list = [value for value in name_list if '.' not in value]

In [23]:
Disease_Group2 = dict(zip(index_conditions, name_list))

In [24]:
def disease_group_name(row):
    key = (row['Unnamed: 2'], row['GHE cause'])
    return Disease_Group2.get(key, '')

In [25]:
df['Disease Group 2'] = df.apply(lambda row: disease_group_name(row), axis = 1)

In [26]:
df = df.dropna(subset=['Unnamed: 5'])

In [27]:
df = df[~df['Unnamed: 5'].str.contains('\.')]

In [28]:
df.drop(columns = ['Unnamed: 2', 'GHE cause', 'Unnamed: 4'], inplace = True)

In [29]:
df.rename(columns = {'Unnamed: 5':'Disease Name'}, inplace = True)

In [30]:
df = df.reindex(columns = ['GHE code', 'Disease Group','Disease Group 2', 'Disease Name'])

#### Add the Disease Group columns

In [52]:
mortality_rates = pd.merge(df,death_rates, on='GHE code')

In [53]:
mortality_rates['Year'].value_counts()

2019    21228
2000    21228
2015    21228
2010    21228
Name: Year, dtype: int64

### Check datatypes and missing values

In [54]:
mortality_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84912 entries, 0 to 84911
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GHE code         84912 non-null  float64
 1   Disease Group    84912 non-null  object 
 2   Disease Group 2  84912 non-null  object 
 3   Disease Name     84912 non-null  object 
 4   Country          84912 non-null  object 
 5   Death Rate       84912 non-null  object 
 6   Year             84912 non-null  object 
dtypes: float64(1), object(6)
memory usage: 5.2+ MB


In [55]:
mortality_rates['Death Rate'] = pd.to_numeric(mortality_rates['Death Rate'], errors = 'coerce')

In [57]:
mortality_rates['Year'] = pd.to_numeric(mortality_rates['Year'], errors = 'coerce')

In [58]:
mortality_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84912 entries, 0 to 84911
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GHE code         84912 non-null  float64
 1   Disease Group    84912 non-null  object 
 2   Disease Group 2  84912 non-null  object 
 3   Disease Name     84912 non-null  object 
 4   Country          84912 non-null  object 
 5   Death Rate       71417 non-null  float64
 6   Year             84912 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 5.2+ MB


In [59]:
mortality_rates.isna().sum()

GHE code               0
Disease Group          0
Disease Group 2        0
Disease Name           0
Country                0
Death Rate         13495
Year                   0
dtype: int64

In [61]:
mortality_rates[mortality_rates['Death Rate'].isna()]

,GHE code,Disease Group,Disease Group 2,Disease Name,Country,Death Rate,Year
327,30.0,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Tuberculosis,Seychelles,NaN,2010
623,30.0,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Tuberculosis,Iceland,NaN,2019
829,40.0,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,STDs excluding HIV,Luxembourg,NaN,2000
1500,100.0,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,HIV/AIDS,Comoros,NaN,2000
1542,100.0,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,HIV/AIDS,Iraq,NaN,2000
...,...,...,...,...,...,...,...
81976,1580.0,Injuries,Unintentional injuries,Natural disasters,Uruguay,NaN,2019
81977,1580.0,Injuries,Unintentional injuries,Natural disasters,Uzbekistan,NaN,2019
81978,1580.0,Injuries,Unintentional injuries,Natural disasters,Vanuatu,NaN,2019
81979,1580.0,Injuries,Unintentional injuries,Natural disasters,Venezuela (Bolivarian Republic of),NaN,2019


In [63]:
mortality_rates = mortality_rates.dropna(subset = ['Death Rate'])

In [66]:
mortality_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71417 entries, 0 to 84911
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GHE code         71417 non-null  float64
 1   Disease Group    71417 non-null  object 
 2   Disease Group 2  71417 non-null  object 
 3   Disease Name     71417 non-null  object 
 4   Country          71417 non-null  object 
 5   Death Rate       71417 non-null  float64
 6   Year             71417 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 4.4+ MB


### Export Dataset

In [67]:
mortality_rates.to_csv('mortality_rates.csv', index = False)